In [1]:
import os

os.environ['NUMBA_NUM_THREADS'] = '8'
os.environ['NUMBA_THREADING_LAYER'] = 'omp' # my TBB version complains
os.environ['NUMBA_ENABLE_AVX'] = '1'


import numpy as np
from pylab import *
from craco.cardcap import CardcapFile
from astropy.io import fits
from craco.card_averager import Averager
from craco.cardcapmerger import CcapMerger
import glob
import numba

from numba import jit,njit,prange


%matplotlib notebook
%load_ext autoreload
%autoreload 2

In [2]:
nt = 64
nbeam = 36
nbl = 465
nant = 30
nfpga = 6
nc = 4*nfpga
npol = 2


In [3]:
def find_instr(func, keyword, sig=0, limit=5):
    count = 0
    for l in func.inspect_asm(func.signatures[sig]).split('\n'):
        if keyword in l:
            count += 1
            print(l)
            if count >= limit:
                break
    if count == 0:
        print('No instructions found')

In [4]:
cardfiles = glob.glob('/data/craco/ban115/craco-python/notebooks/data/SB43128/run3/1934_b07_c01+f?.fits')
assert len(cardfiles) == 6

cfiles = [CardcapFile(f) for f in cardfiles]
merger = CcapMerger(cardfiles)
fid, blk = next(merger.block_iter())

fileblocks = [next(f.packet_iter(nt*4*nbeam)) for f in cfiles]


AssertionError: 

In [ ]:
len(fileblocks[0])

In [ ]:
fb0 = fileblocks[0]
fb0_block = fb0[:nt]

In [ ]:
#numba.set_num_threads(3)
#numba.get_num_threads()

In [ ]:
avg = Averager(nbeam, nant,nc,nt,npol)


In [ ]:
from craco.card_averager import do_accumulate, accumulate_all

for i in range(10):
    do_accumulate(avg.output, avg.rescale_scales, avg.rescale_stats, avg.count, avg.nant, 0,0,fb0_block, 2, 6)
%timeit do_accumulate(avg.output, avg.rescale_scales, avg.rescale_stats, avg.count, avg.nant, 0,0,fb0_block, 2, 6)


In [ ]:
gflops = nbl*npol*nt*(11)/300e-6/1e9
gflops

In [ ]:
do_accumulate.signatures

In [ ]:
find_instr(do_accumulate, 'vadd') # holy crapsky - if there's a vadd instruction, it's SIMD vectorized

In [ ]:
fb0.dtype.isalignedstruct

In [ ]:
#do_accumulate.parallel_diagnostics(level=4)

In [ ]:
do_accumulate.inspect_types()

In [ ]:
print(do_accumulate.inspect_asm(do_accumulate.signatures[0]))

In [ ]:
numba.set_num_threads(8)
from craco.card_averager import accumulate_all
avg.accumulate_all(fileblocks )

In [ ]:
%timeit avg.accumulate_all(fileblocks )

In [ ]:
nthreads = [1,2,4,8]
time_ms = np.array([246, 134, 82, 45])
figure()
fig,ax = subplots(1,2)
ax[0].plot(nthreads, 1/time_ms)
ax[0].set_xlabel('nthreads')
ax[0].set_ylabel('Throughput (per ms)')
ax[1].plot(nthreads, time_ms)
ax[1].set_xlabel('nthreads')
ax[1].set_xlabel('Executution time( ms)')



In [ ]:
#nt = 64
#nbeam = 36
#nbl = 465
#nant = 30
#nfpga = 6
#nc = 4*nfpga
#npol = 2
# gflops per core
gflops=nbeam*npol*nc*nbl*nt*10/74e-3/float(numba.get_num_threads())/1e9
gflops


In [ ]:
n = 1024*32
indtype=np.int16
dtype=np.int32
#dtype=np.int16
a = np.arange(n, dtype=indtype)
b = np.arange(n, dtype=indtype)
c = np.zeros(n, dtype=dtype)


In [ ]:
%timeit np.add(a,b,out=c)

In [ ]:
gflops = n/4.27e-6/1e9
gflops

In [ ]:
@njit(fastmath=True)
def myadd(a,b,c):
    for i in range(len(a)):
        c[i] = a[i]+b[i]
        
myadd(a,b,c)
%timeit myadd(a,b,c)

In [ ]:
find_instr(myadd, 'vadd')

1024 elements is about 2x lower gflops athan 1024x8 elements - I guess there's overheads or prefetching or something. int16 is about 3x faster than float32. Float32 usues 'vadd' with YMM register = AVX-256 (not 512??). But it looks like  5 Gflops/core is about the limit for float32, and maybe 10Gflops/core at int16 is about as good as you'll go.


| func | N | intype | outtype | runtime | gflops | vadd? | 
| ------| --| ----| ---------| ------| ---| ---|
| myadd | N=32*1024 | int32 | int32 | 5.16us |  6.35  | No |
| myadd | N=32*1024 | int16  | int16 | 3.09us | 10.6  | No |
| myadd | N = 32*1024 | float32 | float32 |  6.48 | 5.0  | Yes |
| myadd | N = 32*1024 | int16 | float32 |  7.3 | 4.4 | No |
| myadd | N = 32*1024 | int16 | int32 | 4.27 | 7.6  | No |

In [ ]:
v = np.arange(10, dtype=np.int16) + np.int16((1<<15) - 5)


In [ ]:
v = np.int32(32767)*np.int32(32767)
np.iinfo(np.int32)

In [ ]:
def sim_casics(nant, sigamp):
    cas = 0
    ics = 0
    nant = 30
    N = 1024
    noise = np.random.randn(nant) + 1j*np.random.randn(nant)
    sig = np.ones(nant) + 1j*0
    s = sigamp*sig + noise
    x = np.outer(s, s.T)
    x.shape
    for a1 in range(nant):
        for a2 in range(a1, nant):
            a = np.abs(x[a1, a2])
            if a1 == a2:
                ics += a
            else:
                cas += a*2
    
    return cas,ics

In [ ]:
siglvls = (0, 0.1, 0.3, 1, 3)
niter = 1024*8
results= np.zeros((len(siglvls), niter,2))
nant = 30
for isig, siglvl in enumerate(siglvls):
    results[isig,...] = np.array([sim_casics(nant, siglvl) for i in range(niter)])
        

In [ ]:
cas_rms = results[0,0,:].std()
ics_rms = results[0,1,:].std()
cas_scale = results[:,0,:]/cas_rms
ics_scale = results[:,0,:]/ics_rms
cas_scale.mean(axis=1).shape


In [ ]:


figure()
errorbar(siglvls, cas_scale.mean(axis=1), yerr=cas_scale.std(axis=1), label='cas')
errorbar(siglvls, ics_scale.mean(axis=1), yerr=ics_scale.std(axis=1), label='ics')

legend()